# Exploratory Data Analysis: Solar Activity & Geomagnetic Indices

Dieses Notebook führt eine explorative Datenanalyse (EDA) durch, um die Zusammenhänge zwischen Sonnenaktivität und geomagnetischen Indizes zu untersuchen.


## 1. Imports und Daten laden


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Styling
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Lade Master-Dataset
master = pd.read_csv("../data/processed/master_monthly_merged.csv")
master["date"] = pd.to_datetime(master["date"])

print(f"Dataset Shape: {master.shape}")
print(f"Zeitraum: {master['date'].min()} bis {master['date'].max()}")
print(f"\nSpalten: {list(master.columns)}")
master.head()


## 2. Datenübersicht


In [ ]:
master.describe()


In [ ]:
print("Fehlende Werte:")
print(master.isnull().sum())


## 3. Korrelationsanalyse


In [ ]:
# Pearson-Korrelationen
corr_vars = ["sn", "fluxadjflux", "kp", "ap"]
corr_matrix = master[corr_vars].corr()

print("Korrelationsmatrix:")
print(corr_matrix.round(3))

# Visualisierung
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt=".3f", cmap="coolwarm", center=0,
            square=True, linewidths=1, cbar_kws={"label": "Korrelation"})
ax.set_title("Korrelationsmatrix: Sonnenaktivität ↔ Geomagnetische Indizes", 
             fontsize=13, fontweight="bold", pad=15)
plt.tight_layout()
plt.show()


## 4. Interpretation und Zusammenfassung


In [ ]:
print("="*60)
print("ZUSAMMENFASSUNG DER ERGEBNISSE")
print("="*60)

print("\n1. Hauptkorrelationen:")
print(f"   Sunspots ↔ F10.7:      {corr_matrix.loc['sn', 'fluxadjflux']:.3f}")
print(f"   Sunspots ↔ Kp:        {corr_matrix.loc['sn', 'kp']:.3f}")
print(f"   F10.7 ↔ Kp:           {corr_matrix.loc['fluxadjflux', 'kp']:.3f}")
print(f"   F10.7 ↔ Ap:           {corr_matrix.loc['fluxadjflux', 'ap']:.3f}")

print("\n2. Interpretation:")
print("   - Die Sonnenaktivität zeigt einen klaren 11-Jahres-Zyklus")
print("   - Sunspots und F10.7 korrelieren stark miteinander")
print("   - Beide zeigen einen Zusammenhang mit geomagnetischen Indizes")
print("   - Lag-Features können die Vorhersagekraft verbessern")
